In [36]:
import pandas as pd
import numpy as np

# Betting Rules
- Rank the Teams in the Playoffs from 12 to 1.
- Each time a game is won, you will be assigned points based on what rank you gave the winning team (ex. if Team # 9 wins you get 9 points)



In [48]:
AFC_QUALIFIERS = ['KC', 'NE', 'HOU', 'BAL', 'LAC', 'IND']
afc_stats = pd.read_csv(pd.compat.StringIO(u'''
Tm,W,L,T,W-L%,PF,PA,PD,MoV,SoS,SRS,OSRS,DSRS
NE,11,5,0,.688,436,325,111,6.9,-1.8,5.2,3.1,2.1
MIA,7,9,0,.438,319,433,-114,-7.1,-1.7,-8.8,-3.6,-5.2
BUF,6,10,0,.375,269,374,-105,-6.6,-0.3,-6.9,-6.3,-0.6
NYJ,4,12,0,.250,333,441,-108,-6.8,-1.1,-7.8,-2.0,-5.9
BAL,10,6,0,.625,389,287,102,6.4,0.6,7.0,0.6,6.4
PIT,9,6,1,.594,428,360,68,4.3,1.3,5.6,3.9,1.7
CLE,7,8,1,.469,359,392,-33,-2.1,1.7,-0.3,-1.0,0.6
CIN,6,10,0,.375,368,455,-87,-5.4,2.0,-3.4,0.0,-3.4
HOU,11,5,0,.688,402,316,86,5.4,-1.5,3.8,2.4,1.4
IND,10,6,0,.625,433,344,89,5.6,-2.2,3.4,3.9,-0.6
TEN,9,7,0,.563,310,303,7,0.4,-0.2,0.2,-3.2,3.5
JAX,5,11,0,.313,245,316,-71,-4.4,0.4,-4.0,-8.1,4.0
LAC,12,4,0,.750,428,329,99,6.2,-0.2,6.0,3.0,2.9
KC,12,4,0,.750,565,421,144,9.0,-0.1,8.9,12.6,-3.8
DEN,6,10,0,.375,329,349,-20,-1.3,0.7,-0.5,-3.6,3.1
OAK,4,12,0,.250,290,467,-177,-11.1,1.8,-9.3,-5.2,-4.1
'''), index_col=0)

NFC_QUALIFIERS = ['NO', 'LAR', 'CHI', 'DAL', 'SEA', 'PHI']
nfc_stats = pd.read_csv(pd.compat.StringIO(u'''
Tm,W,L,T,W-L%,PF,PA,PD,MoV,SoS,SRS,OSRS,DSRS
DAL,10,6,0,.625,339,324,15,0.9,0.2,1.1,-1.9,2.9
PHI,9,7,0,.563,367,348,19,1.2,0.5,1.7,0.0,1.8
WAS,7,9,0,.438,281,359,-78,-4.9,-0.1,-4.9,-5.6,0.6
NYG,5,11,0,.313,369,412,-43,-2.7,0.5,-2.2,0.8,-2.9
CHI,12,4,0,.750,421,283,138,8.6,-2.3,6.3,1.5,4.8
MIN,8,7,1,.531,360,341,19,1.2,-0.6,0.6,-1.2,1.8
GB,6,9,1,.406,376,400,-24,-1.5,-1.2,-2.7,0.0,-2.7
DET,6,10,0,.375,324,360,-36,-2.3,-0.8,-3.0,-3.3,0.3
NO,13,3,0,.813,504,353,151,9.4,0.6,10.1,7.9,2.2
ATL,7,9,0,.438,414,423,-9,-0.6,0.4,-0.1,2.5,-2.6
CAR,7,9,0,.438,376,382,-6,-0.4,1.3,0.9,0.1,0.8
TB,5,11,0,.313,396,464,-68,-4.3,1.7,-2.6,2.0,-4.6
LAR,13,3,0,.813,527,384,143,8.9,-0.4,8.5,9.5,-1.1
SEA,10,6,0,.625,428,347,81,5.1,-0.6,4.5,3.0,1.5
SF,4,12,0,.250,342,435,-93,-5.8,0.3,-5.5,-2.5,-3.1
ARI,3,13,0,.188,225,425,-200,-12.5,1.0,-11.5,-9.6,-1.9
'''), index_col=0)


In [53]:
afc_stats.describe().loc['mean']


W         8.06250
L         7.81250
T         0.12500
W-L%      0.50800
PF      368.93750
PA      369.50000
PD       -0.56250
MoV      -0.03750
SoS      -0.03750
SRS      -0.05625
OSRS     -0.21875
DSRS      0.13125
Name: mean, dtype: float64

In [54]:
nfc_stats.describe().loc['mean']

W         7.812500
L         8.062500
T         0.125000
W-L%      0.492437
PF      378.062500
PA      377.500000
PD        0.562500
MoV       0.018750
SoS       0.031250
SRS       0.075000
OSRS      0.200000
DSRS     -0.137500
Name: mean, dtype: float64

# Probabilistic Model
This can be substituded with any predictive model.  We will start with something simple.

SRS is a quality measure of average points scored - average points lost per game.

We will normalize SRS to zero mean and unit varience for the whole league and use this as a proxy for `team_quality`

Now that quality is normalized we can compute the probability of a win as:

```
prior = 0.6
advantage = (cdf(home_team_quality) - cdf(away_team_quality)) / 2
prob_team_1 = prior + advantage
```
the prior of 60% means we give a 10% advantage to the home team 


In [94]:
from sklearn import preprocessing

stats = pd.concat([afc_stats, nfc_stats])
stats['team_quality'] = preprocessing.MinMaxScaler().fit_transform(stats['SRS'].values.reshape(-1, 1))

def home_prb(home, away):
    advantage = 0.5*(stats.loc[home]['team_quality'] - stats.loc[away]['team_quality'])
    return 0.5 + advantage

def away_prb(home, away):
    return 1 - home_prb(home, away)

# generalized probability of team_1 winning if we dont 
# know the location of the game to be played
def mixed_prb(team_1, team_2):
    return (home_prb(team_1, team_2) + away_prb(team_2, team_1)) / 2

In [95]:
stats.loc['NE']['team_quality']

0.773148148148148

In [106]:
from collections import OrderedDict

# Wild Card Round
wildcard = OrderedDict()
def wildcard_round(home, away):
    wildcard[away] = away_prb(home, away)
    wildcard[home] = home_prb(home, away)
    
wildcard_round('HOU', 'IND')
wildcard_round('BAL', 'LAC')
wildcard_round('DAL', 'SEA')
wildcard_round('CHI', 'PHI')


# Division Round
division = OrderedDict()
def division_round(home, team_1, team_2):
    division[home] = wildcard[team_1] * home_prb(home, team_1) + wildcard[team_2] * home_prb(home, team_2)
    division[team_1] = wildcard[team_1] * away_prb(home, team_1)
    division[team_2] = wildcard[team_2] * away_prb(home, team_2)

division_round('KC',  'IND', 'HOU')
division_round('NE',  'LAC', 'BAL')
division_round('LAR', 'SEA', 'DAL')
division_round('NO',  'PHI', 'CHI')

# Championships ... this is one is fucked
champion = OrderedDict()
def championship_round(t1, t2, t3, t4, t5, t6):
    champion[t1] = division[t1] * (
        division[t2] * home_prb(t1, t2) +
        division[t5] * home_prb(t1, t5) +
        division[t4] * home_prb(t1, t4)
    )
    champion[t2] = division[t2] * (
        division[t1] * away_prb(t1, t2) +
        division[t6] * home_prb(t2, t6) +
        division[t3] * home_prb(t2, t3)
    )
    champion[t3] = division[t3] * (
        division[t2] * away_prb(t2, t3) +
        division[t4] * home_prb(t3, t4) +
        division[t5] * home_prb(t3, t5)
    )
    champion[t4] = division[t4] * (
        division[t1] * away_prb(t1, t4) +
        division[t3] * away_prb(t3, t4) +
        division[t6] * home_prb(t4, t6)
    )
    champion[t5] = division[t5] * (
        division[t1] * away_prb(t1, t5) +
        division[t3] * away_prb(t3, t5) +
        division[t6] * home_prb(t5, t6)
    )
    champion[t6] = division[t6] * (
        division[t2] * away_prb(t2, t6) +
        division[t4] * away_prb(t4, t6) +
        division[t5] * away_prb(t5, t6)
    )
    
championship_round(*AFC_QUALIFIERS)
championship_round(*NFC_QUALIFIERS)

# Superbowl
superbowl = OrderedDict()
for team in AFC_QUALIFIERS + NFC_QUALIFIERS:
    superbowl[team] = 0.0
    
potential_matches =  np.transpose([np.tile(AFC_QUALIFIERS, len(NFC_QUALIFIERS)), np.repeat(NFC_QUALIFIERS, len(AFC_QUALIFIERS))])
for match in potential_matches:
    prior = division[match[0]] * division[match[1]]
    superbowl[match[0]] += prior * mixed_prb(match[0], match[1])
    superbowl[match[1]] += prior * (1 - mixed_prb(match[0], match[1]))
    
wildcard = pd.Series(wildcard)
division = pd.Series(division)
champion = pd.Series(champion)
superbowl = pd.Series(superbowl)

In [116]:
all_rounds = pd.concat([wildcard, division, champion, superbowl], axis=1, sort=False)
all_rounds.fillna(0.0)

all_rounds['total_wins'] = all_rounds.sum(axis=1)

all_rounds.sort_values('total_wins')

,0,1,2,3,total_wins
PHI,0.393519,0.120242,0.046671,0.094050,0.654482
DAL,0.421296,0.138482,0.046672,0.104470,0.710921
IND,0.490741,0.182892,0.080829,0.149563,0.904024
HOU,0.509259,0.194509,0.087764,0.162665,0.954196
LAC,0.476852,0.247257,0.118465,0.231960,1.074534
NE,NaN,0.469372,0.216192,0.422950,1.108514
SEA,0.578704,0.235768,0.098016,0.214975,1.127463
BAL,0.523148,0.283372,0.142328,0.278960,1.227809
CHI,0.606481,0.249893,0.123603,0.248678,1.228656
LAR,NaN,0.625750,0.318084,0.686443,1.630277


0.587962962962963

In [120]:
afc_stats

,W,L,T,W-L%,PF,PA,PD,MoV,SoS,SRS,OSRS,DSRS
Tm,,,,,,,,,,,,
NE,11,5,0,0.688,436,325,111,6.9,-1.8,5.2,3.1,2.1
MIA,7,9,0,0.438,319,433,-114,-7.1,-1.7,-8.8,-3.6,-5.2
BUF,6,10,0,0.375,269,374,-105,-6.6,-0.3,-6.9,-6.3,-0.6
NYJ,4,12,0,0.250,333,441,-108,-6.8,-1.1,-7.8,-2.0,-5.9
BAL,10,6,0,0.625,389,287,102,6.4,0.6,7.0,0.6,6.4
PIT,9,6,1,0.594,428,360,68,4.3,1.3,5.6,3.9,1.7
CLE,7,8,1,0.469,359,392,-33,-2.1,1.7,-0.3,-1.0,0.6
CIN,6,10,0,0.375,368,455,-87,-5.4,2.0,-3.4,0.0,-3.4
HOU,11,5,0,0.688,402,316,86,5.4,-1.5,3.8,2.4,1.4
